# Python SDK Installation¶
The Classiq platform has an accompanied Python SDK package that is integrated with the Classiq platform. After registering to the platform, one can install the SDK package.

The package should be installed using pip with:

In [ ]:
!pip install -U classiq

In [1]:
import classiq
classiq.authenticate()

/home/student/.cirq/lib/python3.10/site-packages/classiq/_internals/authentication/token_manager.py:96: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


## Design (SDK) - Optimize (SDK) - Analyze (IDE) - Execute (SDK)¶
Design your hello world quantum algorithm by running the following code in your favorite Python SDK environment (after installing Classiq):

In [2]:
from classiq import *


@qfunc
def main(x: Output[QNum], y: Output[QNum]):

    allocate(4, x)
    hadamard_transform(x)  # creates a uniform superposition
    y |= x**2 + 1

In [3]:
quantum_program = synthesize(create_model(main))

In [4]:
show(quantum_program)

Opening: https://platform.classiq.io/circuit/e12b7e93-e2a1-4acc-b48c-a0d2ca4dc8d1?version=0.42.2


In [5]:
job = execute(quantum_program)
results = job.result()[0].value.parsed_counts
print(results)

[{'x': 11.0, 'y': 122.0}: 149, {'x': 14.0, 'y': 197.0}: 141, {'x': 3.0, 'y': 10.0}: 141, {'x': 9.0, 'y': 82.0}: 141, {'x': 8.0, 'y': 65.0}: 134, {'x': 10.0, 'y': 101.0}: 132, {'x': 6.0, 'y': 37.0}: 130, {'x': 13.0, 'y': 170.0}: 126, {'x': 12.0, 'y': 145.0}: 123, {'x': 7.0, 'y': 50.0}: 122, {'x': 1.0, 'y': 2.0}: 121, {'x': 5.0, 'y': 26.0}: 120, {'x': 0.0, 'y': 1.0}: 120, {'x': 2.0, 'y': 5.0}: 118, {'x': 15.0, 'y': 226.0}: 118, {'x': 4.0, 'y': 17.0}: 112]


In [6]:
write_qmod(create_model(main), "hello_many_worlds")

### Exercise 1: Follow the [Design - The Qmod Language Tutorial](https://nightly.docs.classiq.io/latest/classiq_101/classiq_concepts/design/#concrete-example)

Task: Write within the IDE with the native syntax a quantum algorithm that coherently calculates y= 2*x + 4*z +2, where x and z are quantum numbers represented with 5 qubits each, and they are initialized in a superposition of all possible values.

In [ ]:
#solution in Python
from classiq import Output, QNum, allocate, hadamard_transform, qfunc


@qfunc
def main(x: Output[QNum], y: Output[QNum], z: Output[QNum]):

    allocate(5, x)
    allocate(5, z)
    hadamard_transform(x)  # creates a uniform superposition
    hadamard_transform(z)
    y |= 2*x + 4*z + 2

**continue on the platform**

### Exercise 2: Follow the [Design - Quantum Variables and Functions tutorial](https://nightly.docs.classiq.io/latest/classiq_101/classiq_concepts/design/quantum_variables_and_functions/)

Task: Create another quantum function flip_lsb that flips the least significant bit

### Exercise 3: Follow the [Optimize - The Synthesis Engine tutorial](https://nightly.docs.classiq.io/latest/classiq_101/classiq_concepts/optimize/)

Task: Modify the constraints such that you optimize the circuit for minimal circuit depth using maximum 25 qubits. What are the circuit depth and width you receive? Are they different from the given example?

The task covered in the 'Design - The Qmod Language' tutorial is to create a quantum algorithm that calculates the arithmetic expression **$ y = x^2 + 1 $** in a superposition. The following model written in Qmod (Python) implements the desired task:

In [28]:
from classiq import *


@qfunc
def main(x: Output[QNum], y: Output[QNum]):

    allocate(4, x)
    hadamard_transform(x)  # creates a uniform superposition
    y |= x**2 + 1

In [29]:
# one can always synthesize without any constraints or preferences:
quantum_program = synthesize(create_model(main))

In [30]:
# a constraint is required in this exercise: minimal circuit depth using maximum 25 qubits
quantum_model = create_model(main)
quantum_model_with_constraints = set_constraints(
    quantum_model, Constraints(optimization_parameter="depth", max_width=25)
)

# synthesize with constraints
quantum_model_with_preferences = set_preferences(
    quantum_model_with_constraints,
    Preferences(backend_service_provider="IBM Quantum", backend_name="ibm_brisbane"),
)

quantum_program2 = synthesize(quantum_model_with_preferences)

In [31]:
# check parameters for circuit without contraints
circuit_width = QuantumProgram.from_qprog(quantum_program).data.width
circuit_depth = QuantumProgram.from_qprog(quantum_program).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")

The circuit width is 12 and the circuit_depth is 392


In [32]:
# check parameters for circuit with contraints
circuit_width2 = QuantumProgram.from_qprog(quantum_program2).data.width
circuit_depth2 = QuantumProgram.from_qprog(quantum_program2).transpiled_circuit.depth
print(f"The circuit width is {circuit_width2} and the circuit_depth is {circuit_depth2}")

The circuit width is 21 and the circuit_depth is 1459


### Exercise 4: Follow the [Analyze - Classiq Visualization Tool tutorial](https://nightly.docs.classiq.io/latest/classiq_101/classiq_concepts/analyze/) in the IDE

Task: Export the synthesized quantum program as a LaTeX file, and view it in a LaTeX Editor (it is recommended to use [Overleaf](https://www.overleaf.com/) -  a free, easy to use online LaTeX editor)

### Exercise 5: Follow the [Execute](https://nightly.docs.classiq.io/latest/classiq_101/classiq_concepts/execute/) tutorial

Task A: Adapt the code such that the quantum number x is allocated with 10 qubits. 
Task B: Execute the algorithm with 5096 shots and post process the results from your Python SDK. Plot a graph of all the measured values of x and y with the corresponding axes (make sure you receive the graph of y=x2 +1).

In [37]:
# Implementing task A: allocate 10 qubits to x
from classiq import *


@qfunc
def main(x: Output[QNum], y: Output[QNum]):

    allocate(10, x) #allocate 10 qubits
    hadamard_transform(x)  # creates a uniform superposition
    y |= x**2 + 1

# synthesize algorithm in default settings
# quantum_program = synthesize(create_model(main))


In [38]:
# Implementing task B: Execute with 5096 shots
from classiq.execution import ExecutionPreferences

quantum_model = create_model(main)

quantum_model_with_preferences = set_preferences(
    quantum_model,
    Preferences(backend_service_provider="IBM Quantum", backend_name="ibm_brisbane"),
)


quantum_model_with_execution_preferences = set_execution_preferences(
    quantum_model_with_preferences,
    ExecutionPreferences(
        num_shots=5096, job_name="w24_ex5", random_seed=767
    )
)

In [2]:
import classiq
classiq.authenticate()

### Example on creating Multi-controlled X gate

The multi-control-X applies X gate to one target qubit bit only if the logical AND of all control qubits is satisfied. The multi-control-X function incorporates numerous implementations for the multi-control-X gate, each with a different depth and number of auxiliary qubits

In [2]:
from classiq import (
    Output,
    QArray,
    QBit,
    X,
    allocate,
    control,
    create_model,
    prepare_bell_state,
    qfunc,
)


@qfunc
def main(cntrl: Output[QArray[QBit]], target: Output[QBit]) -> None:
    allocate(7, cntrl)
    allocate(1, target)
    control(ctrl=cntrl, operand=lambda: X(target))

In [3]:
qmod = create_model(main)

In [4]:
from classiq import synthesize, write_qmod

write_qmod(qmod, "mcx_example")
qprog = synthesize(qmod)

In [6]:
from classiq import show
show(qprog)

Opening: https://platform.classiq.io/circuit/e2f85f91-6858-4755-b78a-862491a701dc?version=0.42.2


## w24 Assignment 6.6

**Task A:** Synthesize 3 different implementations of an MCX (multi-control-x) with 5 control qubits and 1 target qubit (you should use the control quantum operation for implementing an MCX, follow this [tutorial](https://docs.classiq.io/latest/explore/functions/function_usage_examples/mcx/mcx_example/) that can be open in the IDE).
One implementation should be optimized for minimized depth, the other for minimized width, and the third somewhere in between (choose yourself what is the maximal width / depth you apply).

Export the 3 implementations as LaTeX files on the hierarchy level that demonstrates the differences between the implementations. Aggregate the implementations in 1 file and export it as a PDF and explain the key differences. (It is recommended to use [Overleaf](https://www.overleaf.com/) -  a free, easy to use online LaTeX editor)

In [7]:
from classiq import (
    Output,
    QArray,
    QBit,
    X,
    allocate,
    control,
    create_model,
    prepare_bell_state,
    qfunc,
)


@qfunc
def main(cntrl: Output[QArray[QBit]], target: Output[QBit]) -> None:
    allocate(5, cntrl)
    allocate(1, target)
    control(ctrl=cntrl, operand=lambda: X(target))

In [8]:
model = create_model(main)

### Implementation 1 optimized for minimized depth

In [10]:
from classiq import *

model1 = set_constraints(
    model, Constraints(optimization_parameter="depth")
)

qmod1 = synthesize(model1)

# check parameters for circuit without contraints
circuit_width = QuantumProgram.from_qprog(qmod1).data.width
circuit_depth = QuantumProgram.from_qprog(qmod1).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")

The circuit width is 8 and the circuit_depth is 34


In [11]:
show(qmod1)

Opening: https://platform.classiq.io/circuit/2f8e7924-fd64-4a16-aacc-db69071c3536?version=0.42.2


### Implementation 2 optimized for minimized width

In [12]:
model2 = set_constraints(
    model, Constraints(optimization_parameter="width")
)

qmod2 = synthesize(model2)

# check parameters for circuit without contraints
circuit_width = QuantumProgram.from_qprog(qmod2).data.width
circuit_depth = QuantumProgram.from_qprog(qmod2).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")

The circuit width is 6 and the circuit_depth is 117


In [13]:
show(qmod2)

Opening: https://platform.classiq.io/circuit/7a8deb95-57b2-4117-b7ba-9ce904013bdb?version=0.42.2


### Implementation 2 optimized for optimal choice

In [16]:
model3 = set_constraints(
    model, Constraints(optimization_parameter="depth", max_width=7)
)

qmod3 = synthesize(model3)

# check parameters for circuit without contraints
circuit_width = QuantumProgram.from_qprog(qmod3).data.width
circuit_depth = QuantumProgram.from_qprog(qmod3).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")

The circuit width is 7 and the circuit_depth is 51


In [17]:
show(qmod3)

Opening: https://platform.classiq.io/circuit/6bc39410-7677-4f3b-ba48-8400509532bf?version=0.42.2


**Task B:** Synthesize 2 different implementations of an MCX (multi-control-x) with 20 control qubits and 1 target qubit. Compare the circuit width and circuit depth required for each implementation.

In [2]:
from classiq import (
    Output,
    QArray,
    QBit,
    X,
    allocate,
    control,
    create_model,
    prepare_bell_state,
    qfunc,
)


@qfunc
def main(cntrl: Output[QArray[QBit]], target: Output[QBit]) -> None:
    allocate(20, cntrl)
    allocate(1, target)
    control(ctrl=cntrl, operand=lambda: X(target))

In [3]:
model = create_model(main)

### Implementation 1 optimized for minimized depth

In [4]:
from classiq import *

model1 = set_constraints(
    model, Constraints(optimization_parameter="depth")
)

qmod1 = synthesize(model1)

# check parameters for circuit without contraints
circuit_width = QuantumProgram.from_qprog(qmod1).data.width
circuit_depth = QuantumProgram.from_qprog(qmod1).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit depth is {circuit_depth}")

The circuit width is 30 and the circuit_depth is 66


In [6]:
show(qmod1)

Opening: https://platform.classiq.io/circuit/7b5661ee-62f9-4291-b5f7-9ddbacc8c5cb?version=0.42.2


### Implementation 2 optimized for minimized width

In [5]:
model2 = set_constraints(
    model, Constraints(optimization_parameter="width")
)

qmod2 = synthesize(model2)

# check parameters for circuit without contraints
circuit_width = QuantumProgram.from_qprog(qmod2).data.width
circuit_depth = QuantumProgram.from_qprog(qmod2).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit depth is {circuit_depth}")

The circuit width is 22 and the circuit depth is 1894


In [7]:
show(qmod2)

Opening: https://platform.classiq.io/circuit/b1e8caed-f37a-475c-aa17-9c1be284a2d6?version=0.42.2
